The conversion of displacements into cartesians nessecitates the use of an iterative procedure called the back transformation.
As with the conversion of forces or gradients, implementation of the pseudo inverse leads to the similar looking result.
$$ \textbf G^{-1}_0 \textbf {B}^T_0 u\Delta q_0 = \Delta x_0$$

Here, the G matrix has dimensions of cartesian not internals as it is calculated with $\textbf B^Tu \textbf B$ where the u matrix has dimesnions of the number of internal coordinates

The iterative back transformation becomes nessecary because the new cartesian geometry is not perfectly eqivalent to the new geometry we stepped to in internals
$$x_0 + \Delta x = x_1$$
To converge on the correct cartesian geometry, we take the error in internals and convert again to cartesians

Call the cartesian geometry converted back to internals as previously shown, $q_x$ and the geometry we calcualted $q_1$ then
$$q_x - q_1 = q_\epsilon$$
We can then resolve our conversian equation having recalulated the nessecary matrices at the new geometry
$$ \textbf G^{-1}_n \textbf {B}_n^T u\Delta q_\epsilon = \Delta x_n$$

To demonsrtrate the need for the back transformation, we'll take a step, in internal coordinates, of 0.2 bohr for the bond lengths and then convert it into cartesians, and back again into internals

In [2]:
# A simple Psi4 input script to compute a Wilson B matrix
# edited with jupyter
# Created by: Rollin A. King
# Date: 7/27/17
# License: GPL v3.0

import numpy as np
import psi4
import sys, os
sys.path.append('os.getcwd()')

mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

# Set some options
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})

In [9]:
mol.update_geometry()
xyz = np.array( mol.geometry() )
print('Cartesian geometry')
print(xyz)
originalxyz = xyz

# Create some internal coordinates for water.
# Numbering starts with atom 0.
from opt_helper import stre,bend
r1 = stre.STRE(0,1)
r2 = stre.STRE(0,2)
theta = bend.BEND(1,0,2)
intcos = [r1,r2,theta]

print("%15s%15s" % ('Coordinate', 'Value'))
for I in intcos:
  print("%15s = %15.8f %15.8f" % (I, I.q(xyz), I.qShow(xyz)))

# displacements to be made in au
dq = np.array( [0.2, 0.2, 0.0], float)

# forces - only used for printing here
fq = np.array( [0.0, 0.0, 0.0], float)

qInitial  = np.zeros( (len(intcos)), float)
for i, intco in enumerate(intcos):
    qInitial[i] = intco.q(xyz)
    
q0 = np.add(qInitial, dq)
print("\nNew Geometry in Internals")
print(q0)


#from optking import displace
#displace.displace(intcos, xyz, dq, fq)

#print('Final Displaced Cartesian geometry')
#print(xyz)

# Check result
#print("%15s%15s" % ('Coordinate', 'Value'))
#for I in intcos:
#  print("%15s = %15.8f %15.8f" % (I, I.q(xyz), I.qShow(xyz)))

Cartesian geometry
[[ 0.          0.         -0.14322583]
 [ 0.         -1.63803697  1.13654891]
 [ 0.          1.63803697  1.13654891]]
     Coordinate          Value
         R(1,2) =      2.07869875      1.10000000
         R(1,3) =      2.07869875      1.10000000
       B(2,1,3) =      1.81514242    104.00000000

New Geometry in Internals
[ 2.27869875  2.27869875  1.81514242]


In [29]:
from opt_helper import intcosMisc
from opt_helper import linearAlgebra
#dq0 = np.zeros(len(intcos), float)
#for i in range (2):
#    dq0[i] = 0.2

#print("Our displacement")
#print(dq0)

#q  = np.zeros( (len(intcos)), float)
#for i, intco in enumerate(intcos):
#    q[i] = intco.q(xyz)

#print("New Internal Geometry")
#print(q)
    
#This is what we should be stepping to    
#qGeomWanted = np.add(q, dq0)
#print("New internal geometry")
#print(qGeomWanted)


#Transforming the step in internals to cartesians

B0 = intcosMisc.Bmat(intcos, xyz)
G0 = np.dot(B0.T, B0)
#the u identity matrix is omittted due to the dimensions
G0inv = linearAlgebra.symmMatInv(G0, True)
u = np.identity(len(intcos))
dx0 = np.dot(G0inv, np.dot(B0.T, np.dot(u, dq)))
dx0mat = np.reshape(dx0, (3, 3))
xyzGeom1 = np.add(dx0mat, originalxyz) #add to the step to the orginal geometry

print("New geometry in catresians")
print(xyzGeom1)

print("\n%15s%15s" % ('Coordinate', 'Value'))
for I in intcos:
  print("%15s = %15.8f %15.8f" % (I, I.q(xyzGeom1), I.qShow(xyzGeom1)))

q1 = np.zeros( (len(intcos)), float)
for i, intco in enumerate(intcos):
    q1[i] = intco.q(xyzGeom1)

qE = np.subtract(q0, q1)      
print("Error in backstep")
print(qE)


print("Now we iterate to minimze the error")
B1 = intcosMisc.Bmat(intcos, xyzGeom1)
G1 = np.dot(B1.T, B1)
#the u identity matrix is omittted due to the dimensions
G1inv = linearAlgebra.symmMatInv(G1, True)
u = np.identity(len(intcos))
dx1 = np.dot(G0inv, np.dot(B1.T, np.dot(u, qE)))
dx1mat = np.reshape(dx1, (3, 3))
xyzGeom2 = np.add(dx1mat, xyzGeom1) #add to the step to the orginal geometry

print("\n%15s%15s" % ('Coordinate', 'Value'))
for I in intcos:
  print("%15s = %15.8f %15.8f" % (I, I.q(xyzGeom2), I.qShow(xyzGeom2)))

q2 = np.zeros( (len(intcos)), float)
for i, intco in enumerate(intcos):
    q2[i] = intco.q(xyzGeom2)
    
qE2 = np.subtract(q2, q0)    
print("New error in step")
print(qE2)



#print ("Displacement in cartesians")
#print(dx0)

#B1 = intcosMisc.Bmat(intcos, xyz)


#dq0 = np.dot(B0, dx0)
#print("Step in Internals from the cartesian step")
#print(dq0)




#dx0mat = np.reshape(dx0, (3, 3))
#Compare the cartesian geometries
#xyzGeom1 = np.add(dx0mat, originalxyz)

#print ("Old geometry")
#print (originalxyz)
#print ("New geometry")
#print(xyzGeom0)

#qGeom1 = np.zeros(len(intcos), float)
#for i in range (len(intcos)):
#    qGeom1[i] = (intcos[i].q(xyzGeom0))
#print ("New geometry with error")    
#print (qGeom1)
#dq1Prime = np.subtract(qGeom1, qInitial) #This is the actual Dq we took
#print ("Acutal step from cartesians")
#print (dq1Prime)

#Dq1 = np.subtract(dq, dq1Prime) #This is the error in Dq

#print ("This is why we need to back transformation")
#print (Dq1)

New geometry in catresians
[[  0.00000000e+00   6.93889390e-18  -2.25314024e-01]
 [  4.98575560e-17  -1.79563912e+00   1.17759301e+00]
 [  0.00000000e+00   1.79563912e+00   1.17759301e+00]]

     Coordinate          Value
         R(1,2) =      2.27869875      1.20583544
         R(1,3) =      2.27869875      1.20583544
       B(2,1,3) =      1.81514242    104.00000000
Error in backstep
[  4.44089210e-16   0.00000000e+00   0.00000000e+00]
Now we iterate to minimze the error

     Coordinate          Value
         R(1,2) =      2.27869875      1.20583544
         R(1,3) =      2.27869875      1.20583544
       B(2,1,3) =      1.81514242    104.00000000
New error in step
[  0.00000000e+00  -4.44089210e-16   0.00000000e+00]
